In [1]:
# Install required libraries
!pip install pyannote.audio transformers librosa pydub speechrecognition nltk textblob
!pip install torchaudio
!pip install vaderSentiment

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.8/897.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.5/828.5 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 864.1/864.1 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [2]:
import torch
from transformers import pipeline
from pyannote.audio import Pipeline
import librosa
import numpy as np
from pydub import AudioSegment
import speech_recognition as sr
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import urllib.request
import os

# Download punkt for nltk sentence tokenizer
import nltk
nltk.download('punkt')


/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\

True

In [3]:
!pip install yt-dlp
!yt-dlp -x --audio-format wav --output "call_audio.%(ext)s" "https://www.youtube.com/watch?v=4ostqJD3Psc"

audio_path = "call_audio.wav"



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.5 MB/s eta 0:00:00
[youtube] Extracting URL: https://www.youtube.com/watch?v=4ostqJD3Psc
[youtube] 4ostqJD3Psc: Downloading webpage
[youtube] 4ostqJD3Psc: Downloading tv simply player API JSON
[youtube] 4ostqJD3Psc: Downloading tv client config
[youtube] 4ostqJD3Psc: Downloading player b66835e2-main
[youtube] 4ostqJD3Psc: Downloading tv player API JSON
[info] 4ostqJD3Psc: Downloading 1 format(s): 251
[download] Sleeping 1.00 seconds as required by the site...
[download] Destination: call_audio.webm
[download] 100% of    1.99MiB in 00:00:00 at 11.61MiB/s
[ExtractAudio] Destination: call_audio.wav
Deleting original file call_audio.webm (pass -k to keep)


In [6]:
from pyannote.audio import Pipeline

HUGGINGFACE_TOKEN = "#Insert your Hugging Face token here when running locally"

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=HUGGINGFACE_TOKEN)


pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`
/usr/local/lib/python3.12/dist-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecap

Model was trained with pyannote.audio 0.0.1, yours is 3.4.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.8.0+cu126. Bad things might happen unless you revert torch to 1.x.


hyperparams.yaml: 0.00B [00:00, ?B/s]

...

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/mean_var_norm_emb.ckpt' -> '/root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["mean_var_norm_emb"] = /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
INFO:speechbrain.utils.fetching:Fetch classifier.ckpt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/classifier.ckpt' -> '/root/.cache/torch/pyannote/speechbrain/classifier.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["classifier"] = /root/.cache/torch/pyannote/speechbrain/classifier.ckpt
INFO:speechbrain.utils.fetching:Fetch label_encoder.txt: Fetching from HuggingFace Hub 'speechbrain/spkrec-ecapa-voxceleb' if not cached


label_encoder.txt: 0.00B [00:00, ?B/s]

DEBUG:speechbrain.utils.fetching:Fetch: Local file found, creating symlink '/root/.cache/huggingface/hub/models--speechbrain--spkrec-ecapa-voxceleb/snapshots/0f99f2d0ebe89ac095bcc5903c4dd8f72b367286/label_encoder.txt' -> '/root/.cache/torch/pyannote/speechbrain/label_encoder.ckpt'
DEBUG:speechbrain.utils.parameter_transfer:Set local path in self.paths["label_encoder"] = /root/.cache/torch/pyannote/speechbrain/label_encoder.ckpt
INFO:speechbrain.utils.parameter_transfer:Loading pretrained files for: embedding_model, mean_var_norm_emb, classifier, label_encoder
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): embedding_model -> /root/.cache/torch/pyannote/speechbrain/embedding_model.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): mean_var_norm_emb -> /root/.cache/torch/pyannote/speechbrain/mean_var_norm_emb.ckpt
DEBUG:speechbrain.utils.parameter_transfer:Redirecting (loading from local path): classifier -> /root/.cac

In [9]:
import librosa
import numpy as np

y, sr = librosa.load("call_audio.wav", sr=None)

frame_length = 2048
hop_length = 512
energy = np.array([
    sum(abs(y[i:i+frame_length]**2))
    for i in range(0, len(y), hop_length)
])

threshold = np.percentile(energy, 75)
voice_segments = energy > threshold

segments = []
start = None
for i, is_voiced in enumerate(voice_segments):
    if is_voiced and start is None:
        start = i * hop_length / sr
    elif not is_voiced and start is not None:
        end = i * hop_length / sr
        segments.append((start, end))
        start = None
if start is not None:
    segments.append((start, len(y) / sr))

print("Detected voice segments (sec):")
for s in segments:
    print(s)


Detected voice segments (sec):
(8.010666666666667, 8.074666666666667)
(8.288, 8.352)
(8.448, 8.512)
(9.045333333333334, 9.088)
(9.312, 9.386666666666667)
(9.568, 9.610666666666667)
(9.792, 9.909333333333333)
(10.410666666666666, 10.464)
(10.549333333333333, 10.592)
(10.709333333333333, 10.773333333333333)
(11.2, 11.274666666666667)
(11.509333333333334, 11.637333333333334)
(11.68, 11.818666666666667)
(11.904, 11.989333333333333)
(12.0, 12.085333333333333)
(12.16, 12.245333333333333)
(12.416, 12.490666666666666)
(12.576, 12.661333333333333)
(12.714666666666666, 12.832)
(14.037333333333333, 14.101333333333333)
(14.261333333333333, 14.272)
(14.368, 14.464)
(14.709333333333333, 14.730666666666666)
(15.530666666666667, 15.594666666666667)
(15.658666666666667, 15.68)
(15.872, 16.042666666666666)
(16.074666666666666, 16.288)
(16.298666666666666, 16.448)
(16.512, 16.533333333333335)
(16.565333333333335, 16.736)
(16.78933333333333, 16.842666666666666)
(16.88533333333333, 17.066666666666666)
(17.

In [10]:
import speech_recognition as sr
from pydub import AudioSegment

audio = AudioSegment.from_wav("call_audio.wav")
recognizer = sr.Recognizer()

transcriptions = []
questions_count = 0

for idx, (start, end) in enumerate(segments):
    segment_audio = audio[start*1000:end*1000]  # pydub uses ms
    filename = f"segment_{idx}.wav"
    segment_audio.export(filename, format="wav")

    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            transcriptions.append(text)
            questions_count += text.count('?')
        except:
            transcriptions.append("")

print(f"Total questions asked: {questions_count}")
print("Sample transcriptions:")
for i, t in enumerate(transcriptions[:5]):
    print(f"Segment {i}: {t}")


Total questions asked: 0
Sample transcriptions:
Segment 0: 
Segment 1: 
Segment 2: 
Segment 3: 
Segment 4: 


In [11]:
merged_segments = []
min_gap = 0.5  # seconds

current_start, current_end = segments[0]
for start, end in segments[1:]:
    if start - current_end <= min_gap:
        current_end = end
    else:
        merged_segments.append((current_start, current_end))
        current_start, current_end = start, end
merged_segments.append((current_start, current_end))

print("Merged voice segments (sec):")
for s in merged_segments:
    print(s)


Merged voice segments (sec):
(8.010666666666667, 8.512)
(9.045333333333334, 9.909333333333333)
(10.410666666666666, 12.832)
(14.037333333333333, 14.730666666666666)
(15.530666666666667, 26.602666666666668)
(27.413333333333334, 30.154666666666667)
(30.890666666666668, 35.232)
(35.733333333333334, 36.565333333333335)
(37.504, 40.36266666666667)
(41.205333333333336, 42.48533333333334)
(43.01866666666667, 49.93066666666667)
(50.69866666666667, 53.13066666666667)
(53.952, 57.824)
(58.45333333333333, 60.117333333333335)
(60.81066666666667, 65.52533333333334)
(66.08, 67.68)
(68.53333333333333, 70.88)
(72.30933333333333, 78.336)
(79.38133333333333, 80.91733333333333)
(81.61066666666666, 82.624)
(83.50933333333333, 86.51733333333334)
(87.12533333333333, 91.712)
(92.40533333333333, 96.10666666666667)
(97.00266666666667, 98.82666666666667)
(99.41333333333333, 103.97866666666667)
(104.85333333333334, 109.19466666666666)
(110.29333333333334, 111.552)
(112.064, 112.71466666666667)
(113.3973333333333

In [12]:
transcriptions = []
questions_count = 0

for idx, (start, end) in enumerate(merged_segments):
    segment_audio = audio[start*1000:end*1000]  # pydub uses ms
    filename = f"merged_segment_{idx}.wav"
    segment_audio.export(filename, format="wav")

    with sr.AudioFile(filename) as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
            transcriptions.append(text)
            questions_count += text.count('?')
        except:
            transcriptions.append("")

print(f"Total questions asked: {questions_count}")
print("Sample transcriptions:")
for i, t in enumerate(transcriptions[:5]):
    print(f"Segment {i}: {t}")


Total questions asked: 0
Sample transcriptions:
Segment 0: thank you
Segment 1: my name is
Segment 2: I have your name is John
Segment 3: thank you
Segment 4: I was just calling about to see how much it would cost to update the map in my car I'd be happy to help you with that state did you receive a mailer from us I did do you need the customer number


In [13]:
total_call_duration = librosa.get_duration(filename="call_audio.wav")

talk_times = []
for start, end in merged_segments:
    talk_times.append(end - start)

total_talk_time = sum(talk_times)

print(f"Total call duration: {total_call_duration:.2f} seconds")
print(f"Total detected talk-time: {total_talk_time:.2f} seconds")
print(f"Talk-time as % of call: {total_talk_time/total_call_duration*100:.2f}%")

longest_monologue = max(talk_times)
print(f"Longest monologue duration: {longest_monologue:.2f} seconds")

# Talk-time ratio (approximate)
ratios = [t / total_talk_time for t in talk_times]
for i, r in enumerate(ratios):
    print(f"Segment {i} talk-time ratio: {r*100:.2f}%")


Total call duration: 122.69 seconds
Total detected talk-time: 86.94 seconds
Talk-time as % of call: 70.86%
Longest monologue duration: 11.07 seconds
Segment 0 talk-time ratio: 0.58%
Segment 1 talk-time ratio: 0.99%
Segment 2 talk-time ratio: 2.78%
Segment 3 talk-time ratio: 0.80%
Segment 4 talk-time ratio: 12.73%
Segment 5 talk-time ratio: 3.15%
Segment 6 talk-time ratio: 4.99%
Segment 7 talk-time ratio: 0.96%
Segment 8 talk-time ratio: 3.29%
Segment 9 talk-time ratio: 1.47%
Segment 10 talk-time ratio: 7.95%
Segment 11 talk-time ratio: 2.80%
Segment 12 talk-time ratio: 4.45%
Segment 13 talk-time ratio: 1.91%
Segment 14 talk-time ratio: 5.42%
Segment 15 talk-time ratio: 1.84%
Segment 16 talk-time ratio: 2.70%
Segment 17 talk-time ratio: 6.93%
Segment 18 talk-time ratio: 1.77%
Segment 19 talk-time ratio: 1.17%
Segment 20 talk-time ratio: 3.46%
Segment 21 talk-time ratio: 5.28%
Segment 22 talk-time ratio: 4.26%
Segment 23 talk-time ratio: 2.10%
Segment 24 talk-time ratio: 5.25%
Segment 25

/tmp/ipython-input-1250678882.py:1: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  total_call_duration = librosa.get_duration(filename="call_audio.wav")


In [14]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

full_text = " ".join(transcriptions)

analyzer = SentimentIntensityAnalyzer()
sentiment_scores = analyzer.polarity_scores(full_text)

if sentiment_scores['compound'] >= 0.05:
    sentiment = "Positive"
elif sentiment_scores['compound'] <= -0.05:
    sentiment = "Negative"
else:
    sentiment = "Neutral"

print(f"Call sentiment: {sentiment}")
print(f"Sentiment scores: {sentiment_scores}")


Call sentiment: Positive
Sentiment scores: {'neg': 0.008, 'neu': 0.77, 'pos': 0.222, 'compound': 0.9933}


In [15]:
summary = f"""
Call Quality Analysis Summary:

- Total call duration: {total_call_duration:.2f} seconds
- Total detected talk-time: {total_talk_time:.2f} seconds
- Talk-time coverage: {total_talk_time/total_call_duration*100:.2f}%
- Longest uninterrupted monologue duration: {longest_monologue:.2f} seconds
- Total questions asked during the call: {questions_count}
- Overall call sentiment: {sentiment}

Actionable Insights:
- The call has high talk-time coverage indicating good engagement.
- No questions were detected; consider training agents to ask more clarifying questions.
- Positive sentiment suggests a friendly tone, maintain this during calls.
- Monitor long monologues and introduce interactive elements to keep customer engaged.

This analysis is based on automated voice activity detection and speech recognition.
"""

print(summary)



Call Quality Analysis Summary:

- Total call duration: 122.69 seconds
- Total detected talk-time: 86.94 seconds
- Talk-time coverage: 70.86%
- Longest uninterrupted monologue duration: 11.07 seconds
- Total questions asked during the call: 0
- Overall call sentiment: Positive

Actionable Insights:
- The call has high talk-time coverage indicating good engagement.
- No questions were detected; consider training agents to ask more clarifying questions.
- Positive sentiment suggests a friendly tone, maintain this during calls.
- Monitor long monologues and introduce interactive elements to keep customer engaged.

This analysis is based on automated voice activity detection and speech recognition.

